In [1]:
import numpy as np

from openforcefield.typing.engines.smirnoff import ForceField
from openforcefield.topology.molecule import Molecule
from openforcefield.topology.topology import Topology

from system.typing.smirnoff import build_smirnoff_map, build_smirnoff_collection
from system.system import System

In [2]:
# Load in a mainline OpenFF forcefield and construct a minimal ethanol topology
parsley = ForceField('openff-1.0.0.offxml')

top =  Topology.from_molecules(Molecule.from_smiles('CCO'))

In [3]:
# Generate a (partial) mapping of topology slots and SMIRNOFF SMIRKS
smirks_map = build_smirnoff_map(forcefield=parsley, topology=top)

smirks_map

{'Constraints': {(0, 3): '[#1:1]-[*:2]',
  (0, 4): '[#1:1]-[*:2]',
  (0, 5): '[#1:1]-[*:2]',
  (1, 6): '[#1:1]-[*:2]',
  (1, 7): '[#1:1]-[*:2]',
  (2, 8): '[#1:1]-[*:2]'},
 'Bonds': {(0, 1): '[#6X4:1]-[#6X4:2]',
  (0, 3): '[#6X4:1]-[#1:2]',
  (0, 4): '[#6X4:1]-[#1:2]',
  (0, 5): '[#6X4:1]-[#1:2]',
  (1, 2): '[#6:1]-[#8:2]',
  (1, 6): '[#6X4:1]-[#1:2]',
  (1, 7): '[#6X4:1]-[#1:2]',
  (2, 8): '[#8:1]-[#1:2]'},
 'Angles': {(0, 1, 2): '[*:1]~[#6X4:2]-[*:3]',
  (0, 1, 6): '[*:1]~[#6X4:2]-[*:3]',
  (0, 1, 7): '[*:1]~[#6X4:2]-[*:3]',
  (1, 0, 3): '[*:1]~[#6X4:2]-[*:3]',
  (1, 0, 4): '[*:1]~[#6X4:2]-[*:3]',
  (1, 0, 5): '[*:1]~[#6X4:2]-[*:3]',
  (1, 2, 8): '[*:1]-[#8:2]-[*:3]',
  (2, 1, 6): '[*:1]~[#6X4:2]-[*:3]',
  (2, 1, 7): '[*:1]~[#6X4:2]-[*:3]',
  (3, 0, 4): '[#1:1]-[#6X4:2]-[#1:3]',
  (3, 0, 5): '[#1:1]-[#6X4:2]-[#1:3]',
  (4, 0, 5): '[#1:1]-[#6X4:2]-[#1:3]',
  (6, 1, 7): '[#1:1]-[#6X4:2]-[#1:3]'},
 'ProperTorsions': {(0, 1, 2, 8): '[#6X4:1]-[#6X4:2]-[#8X2H1:3]-[#1:4]',
  (2, 1, 0, 3): '

In [4]:
# This is not efficient at the moment, will take a minute or so to run
smirnoff_collection = build_smirnoff_collection(forcefield=parsley, smirks_map=smirks_map)

In [5]:
# This is like the smirnoff_collection in the other notebook, but encompasses more parameters
# (all vdW and bond parameters in the force field). You can inspect, for example, the list
# of all SMIRKS patterns of bonds
smirnoff_collection.handlers['Bonds'].potentials.keys()

dict_keys(['[#6X4:1]-[#6X4:2]', '[#6:1]-[#8:2]', '[#6X4:1]-[#1:2]', '[#8:1]-[#1:2]'])

In [6]:
# Or look up a specific parameter within a particlar bond
smirnoff_collection.handlers['Bonds'].potentials['[#6X4:1]-[#1:2]'].parameters['k']

758.0931772913 <Unit('kilocalorie / angstrom ** 2 / mole')>

In [7]:
# Can build up a system object using the data we have
ethanol_system = System(
    potential_collection=smirnoff_collection,
    potential_map=smirks_map,
    topology=top,
    positions=np.zeros(shape=(top.n_topology_atoms, 3)),
    box=np.zeros((3, 3)),
)

/Users/mwt/software/miniconda3/envs/openff-system/lib/python3.7/site-packages/numpy/core/_asarray.py:85: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return array(a, dtype, copy=False, order=order)


In [8]:
# And using the SMIRKS map (which should be merged into the System at some point), we can
# find the SMIRKS that was assigned to an arbitrary bond, say index 4
bond_key = tuple([a.topology_atom_index for a in ethanol_system.topology.bond(4).atoms])
smirks_map['Bonds'][bond_key]

'[#6X4:1]-[#1:2]'

In [9]:
# And then using that SMIRKS as a "key", we can look up its associated potential
ethanol_system.potential_collection['Bonds'].potentials[smirks_map['Bonds'][bond_key]]

ParametrizedAnalyticalPotential(name='b83', smirks='[#6X4:1]-[#1:2]', expression='1/2*k*(length-length_0)**2', independent_variables={'length_0'}, parameters={'k': <Quantity(758.0931772913, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(1.092888378383, 'angstrom')>})